In [1]:
import pandas as pd
import numpy as np
import re
print(pd.__version__)
print(np.__version__)

0.24.2
1.16.2


In [2]:
# 크롤링해 온 국민청원 데이터를 판다스를 통해 읽어온다.
petitions = pd.read_csv('petition.csv',
                        parse_dates=['start', 'end'])
# 데이터의 크기가 어느정도인지 본다.
petitions.shape

(377756, 8)

In [3]:
df=petitions

In [4]:
df.isnull().sum()

article_id    0
start         0
end           0
answered      0
votes         0
category      0
title         0
content       1
dtype: int64

In [5]:
df.shape

(377756, 8)

In [6]:
df.isnull().values.any()

True

In [7]:
df=df.dropna()

In [8]:
df.shape

(377755, 8)

In [9]:
# 전체 데이터 중 투표가 500건 이상인 데이터를 기준으로 가져옵니다. 
# 아웃라이어 데이터 제거를 위해 20만건 이상 데이터도 제거합니다.
petition_remove_outlier = df.loc[(df['votes'] < 1) | 
                                        (df['votes'] > 100)]
petition_remove_outlier.shape

(71416, 8)

In [10]:
df = petition_remove_outlier.copy()

In [11]:
df.describe()

,article_id,answered,votes
count,71416.000000,71416.000000,7.141600e+04
mean,158231.999132,0.000406,7.460335e+02
std,146795.771474,0.020147,1.111777e+04
min,23.000000,0.000000,0.000000e+00
25%,34064.500000,0.000000,0.000000e+00
50%,97239.500000,0.000000,0.000000e+00
75%,271198.250000,0.000000,0.000000e+00
max,492036.000000,1.000000,1.192049e+06


In [12]:
# 기본값을 0으로 세팅, 새로운 컬럼 생성
df['votes_pos_neg'] = 0

In [13]:
# 평균 투표수를 구해 옵니다.
votes_mean = df['votes'].mean()
votes_mean

746.03350789739

In [14]:
# 투표수가 평균을 넘으면 1로 다시 세팅합니다.
df['votes_pos_neg'] = (df['votes'] > 1) == 1

In [15]:
df['votes_pos_neg'].dtypes

dtype('bool')

In [16]:
# 타입을 boolean 에서 int로 변경해 줍니다.
df['votes_pos_neg'] = df['votes_pos_neg'].astype(int)

In [17]:
df[['votes', 'votes_pos_neg']].head()

,votes,votes_pos_neg
2,0,0
4,0,0
7,2137,1
10,0,0
13,679,1


In [18]:
import re
p = re.compile('.*(짱깨).*')
type(p)

re.Pattern

In [19]:
# 관심사별로 텍스트 데이터를 학습시키기 위한 목적도 있지만 전체 텍스트를 돌렸을 때 빠르게 결과를 얻기 위해서 샘플링 하려는 의도도 있습니다.
p = r'.*(짱깨).*'
finance = df[df['title'].str.match(p) |
           df['content'].str.match(p, flags=re.MULTILINE)]
finance.shape

(9, 9)

In [20]:
finance["content"]

25202     일본이 짱깨를 수십만밖에 죽이지 못한 것을 두고 우리는 나쁜 짓이라 욕했지만\n모택...
86840                                         짱깨좀 어떻게 해봐요ㅜㅜ
198026                          제주도는 이미 짱깨한테 먹혔습니다\n정화시켜주세요
217168    그냥 솔직히 말합시다.\n어차피 우리가 최저시급이 올라 그리고 업무 근무 시간도 크...
223298    세금은 세금대로 걷어서 돌아오는 것도 없는 북한에, 통일 되더라도 안정되기 전까진 ...
225595    저는 어느 정당 지지자도 아니지만 문재인 대통령의 정책과 행보들을 보면 정말 탄핵해...
242989    기존 청원 일주일 남아서 다시 올립니다\n탄핵 청원 동의 및 공유 부탁드려요 우리나...
269915    세금은 세금대로 걷어서 돌아오는 것도 없는 북한에, 통일 되더라도 안정되기 전까진 ...
355443    짱깨노래소개코너? 그렇게해서 우리한테 얻는 이득이뭔데요?\n짱깨들한테 그렇게 당하고...
Name: content, dtype: object

In [21]:
finance["content"][25202]

'일본이 짱깨를 수십만밖에 죽이지 못한 것을 두고 우리는 나쁜 짓이라 욕했지만\\n모택동은 같은 나라 사람이면서 짱깨를 수억이나 죽이고 굶기고 탄압했다\\n세상에서 짱꺠를 가장 많이 죽인 사람이 누군지 아는가? 바로 짱꺠인 모택동이다\\n짱깨 하고는 상종을 말자\\n짱깨들은 기술도 후달려서 맨날 미국꺼 뺏겨다 쓴다 그래도 좀 모자르다는 평을 듣는다\\n소련도 망했다 근데 짱깨같은 클라스가 대체 어딜봐서 멸족을 면할 클라스인가\\n공산당이 제일 통치기구인 나라가 뭐가 두렵나\\n짱깨돈을 탐하다가 면치못하게 될 꼴이 동남아다\\n동남아는 화교경제에 피말리듯이 뒤쳐져서 아직도 그 동네에선 짱깨가 탑이다\\n이런 짱깨하고 엮이고 싶어서 발정이난 문재인은 동남아가서 촌장이나 할 까말까한 수준이다\\n문재인을 동남아에 농촌 면장으로 수출해버리자'

In [22]:
p = r'.*(개새끼|새끼|미친).*'
df.loc[df["content"].str.match(p)]

,article_id,start,end,answered,votes,category,title,content,votes_pos_neg
42,65,2017-08-20,2017-11-18,0,285,육아/교육,기간제교사 정규직 및 무기계약직전환 반대합니다,"저는 영어전문강사, 스포츠전문강사를 무기계약직 혹은 정규직으로 전환하는 것에 반대합...",1
55,78,2017-08-20,2017-08-27,0,0,교통/건축/국토,아파트 비리와 맞서다 불법 해임된 아파트 회장,청와대 청원서\n아파트 비리와 맞선 입주자 회장\nFact\n1. 아파트 비리와 타...,0
56,79,2017-08-20,2017-09-04,0,0,교통/건축/국토,전국민의 70%가 살고 있는 아파트 비리 반드시 정부의 정책적 대처가 필요합니다.,청와대 청원서\n아파트 비리와 맞선 입주자 회장\nFact\n1. 아파트 비리와 타...,0
365,392,2017-08-24,2017-09-23,0,0,교통/건축/국토,부동산정책 제안(규제대상자 기준선정),안녕하세요. 부동산정책 제안합니다.\n8.2 부동산 정책의 규제대상자 선정기준을 제...,0
753,781,2017-08-29,2017-09-05,0,0,기타,노래불러주세요,한 이삼십분 작성한거 같은데\n게시판에 없네요.ㅠㅠ\n뭐 똑같은 내용 쓰기는 제가 ...,0
1692,1725,2017-09-03,2017-09-10,0,0,저출산/고령화대책,불조리하게 돌아가는 노노케어,"노노케어,\n일자리가 필요한 건강한 노인이 혼자사는 불편한 노인을 돕는다.\n취지는...",0
2633,2672,2017-09-04,2017-12-03,0,0,교통/건축/국토,운전자 스티커 통일안,"운전을 하다 보면 차량에 초보운전이라는 말은 안써져 있고 '직진만 10년째' , '...",0
2694,2733,2017-09-04,2017-12-03,0,0,인권/성평등,청소년 보호법 폐지,갓 태어난 90일둥이 아이를 둔 초보 아빠입니다.\n오늘 사건을 보니 불안해서 내 ...,0
2810,2851,2017-09-04,2017-09-11,0,0,육아/교육,제발 청법 폐지해주세요..,이제 5살과 갓돌지난 아이를 키우고 있는 엄마입니다..\n이런 기사를 접할때마다 내...,0
3304,3345,2017-09-04,2017-12-03,0,0,미래,청소년법이 무엇입니까?,청소년법을 왜 우리는 믿고 따라야 합니까?\n청소년을 위한 법이지 그것이 악용되는것...,0


In [23]:
df["content"][152677]

'그만 나왔으면 좋겠다~올해도 지겹게 보겠군!\\n넘~자주나오니깐 영화감상에 몰입이 안되네~ㅠㅠ\\n제발~방송피디 영화 감독 제작자들은 인간성 좋은배우 캐스팅좀 하기를~요구합니다~ㅠㅠ문화예절 망칩니다\\n대한민국 감초 유명 배우 이경영을 고소했다.\\n지금부터 12년전 2006년 5월3일에 사건은 발생했다.그이후 그는 선배도 아니고 그냥 양아치 인간으로 18년의 인연을 끊었다.\\n이제 그는 선배도 아니고 생양아치일 뿐이다\\n배고픈 무명배우 후배를 초대하여 지인들 술값을 안낸다고 화를 내고 나를 일방적 폭행 수많은 치욕스런모욕 재물손괴 등 피해주어 의정부지방법원 고양지원의 벌금 100만원의 형사처벌 받았다.\\n당시 목격자 대리기사가 112신고하자  조폭후배들까지 동원해서 나에게 협박을 신변위협을 한 사실도 있었다.\\n형사 확정 판결난 이후 치료비와 인격모욕 등 대하여 정식사과 거부하여 결국에 변호사 선임하여 손해배상소송 제기 승소했지만 12년동안 사과도 않고 병원치료비 손해배상 책임도 무시하는 쓰레기 유명배우 연기자 이다\\n아무리 대한민국 썩은 무능한 법이지만  사법부 판결까지 개무시하는 파렴치한 유명배우 이경* 한국영화 감초 다작출연 조연배우 인데ㅡ이런 악질 비양심 인간을 여러분들은 어떻게 생각합니까? ㅠ\\n길고긴 12년동안 인내한 나의 한계는 더이상은 그인간을 용서 할수가 없다!\\n내가 그당시 선배라는 이유만으로 참지 않았으면 그는 그날 현장에서 즉사해서 죽었을 것이다!\\n사건은 2006년5월3일 저녁10시45분에 일산에서 자신의 "돈주황"삼겹살 식당 개업식에 초대해서 방문하였는데 이경* 악질인간이 자신의 20여명 지인들 술값을 나한테 대신 내라고 요구하여\\n저는 "무명배우 후배가 돈이 어디있냐" 고 거부하니 "카드도 된다"며 카드결재를 강요 하였고 이에 후배는 황당 실망하자 갑자기 다혈질로 변해 인격폭언 인격모욕하고 "너 가라 개새끼야"고 "앞으로 연락하지마" 라고 하여 이에 항의하자\\n지인들이 말리는데도 내얼굴과 머리뺨 수차레 때리고 "

In [24]:
type(p)

str

In [25]:
# 금칙어 리스트를 만들어서 해단 str에 금칙어가 있는지 판단
# 판단하고 어떤 금칙어가 있었는지도 print

In [26]:
import re
p = re.compile('.*(개새끼|새끼|미친).*')
type(p)

re.Pattern

In [27]:
p.match(df["content"][152677])

<re.Match object; span=(0, 3216), match='그만 나왔으면 좋겠다~올해도 지겹게 보겠군!\\n넘~자주나오니깐 영화감상에 몰입이 안되네>

In [28]:
result=p.findall(df["content"][152677])

In [29]:
print(result)

['새끼']


In [30]:
p.search(df["content"][152677])

<re.Match object; span=(0, 3216), match='그만 나왔으면 좋겠다~올해도 지겹게 보겠군!\\n넘~자주나오니깐 영화감상에 몰입이 안되네>

In [31]:
import re
p = re.compile('.*(개새끼|새끼|미친).*')
type(p)

re.Pattern

In [32]:
result = p.finditer("life is too short")
print(result)

In [33]:
for r in result:
    print(r)

In [34]:
import re
  
pattern = '\d+'
text = '10, 20, 30, 40'
 
r = re.compile(pattern)
 
results = r.findall(text)
print(results)

['10', '20', '30', '40']


In [35]:
import re
 
pattern = 'is'
text = 'This is a book'
 
r = re.compile(pattern)
 
match = r.match(text)
print(match)

None


In [36]:
from konlpy.tag import Twitter

In [37]:
twitter = Twitter()

C:\Users\hyunho\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [38]:
print(twitter.nouns(u'유일하게 항공기 체계 종합개발 경험을 갖고 있는 KAI는'))

['항공기', '체계', '종합', '개발', '경험']


In [39]:
type(twitter.nouns(u'유일하게 항공기 체계 종합개발 경험을 갖고 있는 KAI는'))

list

In [40]:
twitter.nouns(u'유일하게 항공기 체계 종합개발 경험을 갖고 있는 KAI는')[0]

'항공기'

In [41]:
twitter.nouns(u'유일하게 항공기 체계 종합개발 경험을 갖고 있는 KAI는')[1]

'체계'

In [42]:
print(twitter.pos(u'이것도 되나욬ㅋㅋ', norm=True, stem=True))

[('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되다', 'Verb'), ('ㅋㅋ', 'KoreanParticle')]


In [43]:
results = []

In [44]:
df["content"][25202]

'일본이 짱깨를 수십만밖에 죽이지 못한 것을 두고 우리는 나쁜 짓이라 욕했지만\\n모택동은 같은 나라 사람이면서 짱깨를 수억이나 죽이고 굶기고 탄압했다\\n세상에서 짱꺠를 가장 많이 죽인 사람이 누군지 아는가? 바로 짱꺠인 모택동이다\\n짱깨 하고는 상종을 말자\\n짱깨들은 기술도 후달려서 맨날 미국꺼 뺏겨다 쓴다 그래도 좀 모자르다는 평을 듣는다\\n소련도 망했다 근데 짱깨같은 클라스가 대체 어딜봐서 멸족을 면할 클라스인가\\n공산당이 제일 통치기구인 나라가 뭐가 두렵나\\n짱깨돈을 탐하다가 면치못하게 될 꼴이 동남아다\\n동남아는 화교경제에 피말리듯이 뒤쳐져서 아직도 그 동네에선 짱깨가 탑이다\\n이런 짱깨하고 엮이고 싶어서 발정이난 문재인은 동남아가서 촌장이나 할 까말까한 수준이다\\n문재인을 동남아에 농촌 면장으로 수출해버리자'

In [45]:
text=df["content"][25202]

In [46]:
results=[]

In [47]:
text='야 이 최현호 미친놈아 프로젝트는 다 하고 노는거냐. 제정신이야. 이 개새끼야 어 열심히 해야지 왜 롤을 하는거야. 롤도 못하는 정신나간 새끼. 시발 미친놈 병신 머저리야. 너는 정말 대가리에 똥만 찼어. 이런 빌어먹을 자식. 내가 나한테 욕하다니 수치스럽다.'

In [48]:
malist = twitter.pos(text, norm=True, stem=True)

In [49]:
for word in malist:
            # 어미/조사/구두점 등은 대상에서 제외 
            if not word[1] in ["Josa", "Eomi", "Punctuation"]:
                results.append(word[0])

In [50]:
type(results)

list

In [51]:
results

['야',
 '이',
 '최현호',
 '미치다',
 '놈',
 '프로젝트',
 '다',
 '하다',
 '놓다',
 '제',
 '정신',
 '이',
 '개새끼',
 '열심히',
 '하다',
 '왜',
 '롤',
 '하다',
 '롤',
 '못',
 '하다',
 '정신',
 '나가다',
 '새끼',
 '시발',
 '미치다',
 '놈',
 '병신',
 '머',
 '저리',
 '너',
 '정말',
 '대가리',
 '똥',
 '차다',
 '이렇다',
 '빌어먹을',
 '자식',
 '내',
 '나',
 '욕',
 '하다',
 '수치',
 '스럽다']

In [52]:
# 2. 비속어 먼저 체크
import re
p = re.compile('.*(개새끼|새끼|미친|병신|대가리).*')

In [53]:
bad=[]

In [54]:
for mal in results:
    #print(mal)
    for r in p.findall(mal):
        print(r)
        bad.append(r)

새끼
새끼
병신
대가리


In [55]:
bad

['새끼', '새끼', '병신', '대가리']

In [56]:
# 1. 중복된 내용인지 체크
text=df["content"][25202]

In [57]:
df.loc[df["content"]==text]

,article_id,start,end,answered,votes,category,title,content,votes_pos_neg
25202,26252,2017-11-05,2017-12-05,0,0,정치개혁,문재인 대통령을 수출합시다,일본이 짱깨를 수십만밖에 죽이지 못한 것을 두고 우리는 나쁜 짓이라 욕했지만\n모택...,0


In [58]:
polarity=pd.read_csv('lexicon\polarity.csv')

In [59]:
polarity

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
0,가*/JKS,1,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
1,가*/JKS;있/VV,1,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
2,가*/JKS;있/VV;었/EP,1,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
3,가*/VV,3,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
4,가*/VV;ㄴ다*/EF,1,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
5,가/JKC,17,0.000000,0.470588,0.235294,0.000000,0.294118,NEG,0.470588
6,가/JKC;되/VV,11,0.000000,0.363636,0.272727,0.000000,0.363636,NEG,0.363636
7,가/JKC;되/VV;ㄴ/ETM,2,0.000000,0.000000,1.000000,0.000000,0.000000,NEUT,1.000000
8,가/JKC;되/VV;ㄹ/ETM,1,0.000000,0.000000,1.000000,0.000000,0.000000,NEUT,1.000000
9,가/JKC;되/VV;어/EC,2,0.000000,1.000000,0.000000,0.000000,0.000000,NEG,1.000000


In [60]:
polarity["ngram"]

0                     가*/JKS
1                가*/JKS;있/VV
2           가*/JKS;있/VV;었/EP
3                      가*/VV
4               가*/VV;ㄴ다*/EF
5                      가/JKC
6                 가/JKC;되/VV
7           가/JKC;되/VV;ㄴ/ETM
8           가/JKC;되/VV;ㄹ/ETM
9            가/JKC;되/VV;어/EC
10        가/JKC;되/VV;어야지요/EF
11              가/JKC;아니/VCN
12        가/JKC;아니/VCN;ㄴ가/EC
13         가/JKC;아니/VCN;면/EC
14       가/JKC;아니/VCN;ㅂ니다/EF
15                     가/JKS
16              가/JKS;가능/NNG
17        가/JKS;가능/NNG;하/XSA
18              가/JKS;가장/MAG
19       가/JKS;가장/MAG;많이/MAG
20        가/JKS;가장/MAG;어렵/VA
21                가/JKS;굵/VA
22                가/JKS;그/MM
23         가/JKS;그/MM;동안/NNG
24              가/JKS;그리/MAG
25         가/JKS;그리/MAG;밝/VA
26              가/JKS;극대/NNG
27        가/JKS;극대/NNG;화/XSN
28               가/JKS;꺾이/VV
29               가/JKS;꽤/MAG
                ...         
16332                  희박/XR
16333                 희생/NNG
16334           희생/NNG;하/XSV
16335         

In [61]:
# 모든 형태소 받기
results=[]

In [62]:
for word in malist:
            results.append(word[0])

In [63]:
results

['야',
 '이',
 '최현호',
 '미치다',
 '놈',
 '아',
 '프로젝트',
 '는',
 '다',
 '하다',
 '놓다',
 '.',
 '제',
 '정신',
 '이야',
 '.',
 '이',
 '개새끼',
 '야',
 '어',
 '열심히',
 '하다',
 '왜',
 '롤',
 '을',
 '하다',
 '.',
 '롤',
 '도',
 '못',
 '하다',
 '정신',
 '나가다',
 '새끼',
 '.',
 '시발',
 '미치다',
 '놈',
 '병신',
 '머',
 '저리',
 '야',
 '.',
 '너',
 '는',
 '정말',
 '대가리',
 '에',
 '똥',
 '만',
 '차다',
 '.',
 '이렇다',
 '빌어먹을',
 '자식',
 '.',
 '내',
 '가',
 '나',
 '한테',
 '욕',
 '하다',
 '수치',
 '스럽다',
 '.']

In [64]:
polarity["ngram"]

0                     가*/JKS
1                가*/JKS;있/VV
2           가*/JKS;있/VV;었/EP
3                      가*/VV
4               가*/VV;ㄴ다*/EF
5                      가/JKC
6                 가/JKC;되/VV
7           가/JKC;되/VV;ㄴ/ETM
8           가/JKC;되/VV;ㄹ/ETM
9            가/JKC;되/VV;어/EC
10        가/JKC;되/VV;어야지요/EF
11              가/JKC;아니/VCN
12        가/JKC;아니/VCN;ㄴ가/EC
13         가/JKC;아니/VCN;면/EC
14       가/JKC;아니/VCN;ㅂ니다/EF
15                     가/JKS
16              가/JKS;가능/NNG
17        가/JKS;가능/NNG;하/XSA
18              가/JKS;가장/MAG
19       가/JKS;가장/MAG;많이/MAG
20        가/JKS;가장/MAG;어렵/VA
21                가/JKS;굵/VA
22                가/JKS;그/MM
23         가/JKS;그/MM;동안/NNG
24              가/JKS;그리/MAG
25         가/JKS;그리/MAG;밝/VA
26              가/JKS;극대/NNG
27        가/JKS;극대/NNG;화/XSN
28               가/JKS;꺾이/VV
29               가/JKS;꽤/MAG
                ...         
16332                  희박/XR
16333                 희생/NNG
16334           희생/NNG;하/XSV
16335         

In [65]:
polarity.loc[2]

ngram        가*/JKS;있/VV;었/EP
freq                        1
COMP                        0
NEG                         0
NEUT                        0
None                        0
POS                         1
max.value                 POS
max.prop                    1
Name: 2, dtype: object

In [66]:
polarity

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
0,가*/JKS,1,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
1,가*/JKS;있/VV,1,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
2,가*/JKS;있/VV;었/EP,1,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
3,가*/VV,3,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
4,가*/VV;ㄴ다*/EF,1,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
5,가/JKC,17,0.000000,0.470588,0.235294,0.000000,0.294118,NEG,0.470588
6,가/JKC;되/VV,11,0.000000,0.363636,0.272727,0.000000,0.363636,NEG,0.363636
7,가/JKC;되/VV;ㄴ/ETM,2,0.000000,0.000000,1.000000,0.000000,0.000000,NEUT,1.000000
8,가/JKC;되/VV;ㄹ/ETM,1,0.000000,0.000000,1.000000,0.000000,0.000000,NEUT,1.000000
9,가/JKC;되/VV;어/EC,2,0.000000,1.000000,0.000000,0.000000,0.000000,NEG,1.000000


In [67]:
text

'일본이 짱깨를 수십만밖에 죽이지 못한 것을 두고 우리는 나쁜 짓이라 욕했지만\\n모택동은 같은 나라 사람이면서 짱깨를 수억이나 죽이고 굶기고 탄압했다\\n세상에서 짱꺠를 가장 많이 죽인 사람이 누군지 아는가? 바로 짱꺠인 모택동이다\\n짱깨 하고는 상종을 말자\\n짱깨들은 기술도 후달려서 맨날 미국꺼 뺏겨다 쓴다 그래도 좀 모자르다는 평을 듣는다\\n소련도 망했다 근데 짱깨같은 클라스가 대체 어딜봐서 멸족을 면할 클라스인가\\n공산당이 제일 통치기구인 나라가 뭐가 두렵나\\n짱깨돈을 탐하다가 면치못하게 될 꼴이 동남아다\\n동남아는 화교경제에 피말리듯이 뒤쳐져서 아직도 그 동네에선 짱깨가 탑이다\\n이런 짱깨하고 엮이고 싶어서 발정이난 문재인은 동남아가서 촌장이나 할 까말까한 수준이다\\n문재인을 동남아에 농촌 면장으로 수출해버리자'

In [68]:
from konlpy.tag import Kkma
kkma=Kkma()

In [69]:
#pos는 형태소를 의미
train=kkma.pos(text)

In [70]:
train

[('일본', 'NNG'),
 ('이', 'JKS'),
 ('짱', 'NNG'),
 ('깨', 'NNG'),
 ('를', 'JKO'),
 ('수십만', 'NR'),
 ('밖에', 'JX'),
 ('죽이', 'VV'),
 ('지', 'ECD'),
 ('못하', 'VX'),
 ('ㄴ', 'ETD'),
 ('것', 'NNB'),
 ('을', 'JKO'),
 ('두', 'VV'),
 ('고', 'ECE'),
 ('우리', 'NP'),
 ('는', 'JX'),
 ('나쁘', 'VA'),
 ('ㄴ', 'ETD'),
 ('짓', 'NNG'),
 ('이', 'VCP'),
 ('라', 'ECD'),
 ('욕하', 'VV'),
 ('었', 'EPT'),
 ('지만', 'ECE'),
 ('\\', 'SW'),
 ('n', 'OL'),
 ('모택동', 'NNG'),
 ('은', 'JX'),
 ('같', 'VA'),
 ('은', 'ETD'),
 ('나라', 'NNG'),
 ('사람', 'NNG'),
 ('이', 'VCP'),
 ('면서', 'ECE'),
 ('짱', 'MAG'),
 ('깨', 'NNG'),
 ('를', 'JKO'),
 ('수억', 'NR'),
 ('이나', 'JX'),
 ('죽이', 'VV'),
 ('고', 'ECE'),
 ('굶기', 'VV'),
 ('고', 'ECE'),
 ('탄압', 'NNG'),
 ('하', 'XSV'),
 ('었', 'EPT'),
 ('다', 'ECS'),
 ('\\', 'SW'),
 ('n', 'OL'),
 ('세상', 'NNG'),
 ('에서', 'JKM'),
 ('짱꺠', 'UN'),
 ('를', 'JKO'),
 ('가장', 'MAG'),
 ('많이', 'MAG'),
 ('죽이', 'VV'),
 ('ㄴ', 'ETD'),
 ('사람', 'NNG'),
 ('이', 'JKS'),
 ('누', 'NNG'),
 ('군지', 'UN'),
 ('알', 'VV'),
 ('는가', 'EFQ'),
 ('?', 'SF'),
 ('바로', 'MAG'),
 (

In [71]:
from konlpy.tag import Hannanum
hannanum=Hannanum()
list=[]

In [72]:
type(list)

list

In [73]:
for word,tag in train:
    list.append(word+'/'+tag)

In [74]:
list

['일본/NNG',
 '이/JKS',
 '짱/NNG',
 '깨/NNG',
 '를/JKO',
 '수십만/NR',
 '밖에/JX',
 '죽이/VV',
 '지/ECD',
 '못하/VX',
 'ㄴ/ETD',
 '것/NNB',
 '을/JKO',
 '두/VV',
 '고/ECE',
 '우리/NP',
 '는/JX',
 '나쁘/VA',
 'ㄴ/ETD',
 '짓/NNG',
 '이/VCP',
 '라/ECD',
 '욕하/VV',
 '었/EPT',
 '지만/ECE',
 '\\/SW',
 'n/OL',
 '모택동/NNG',
 '은/JX',
 '같/VA',
 '은/ETD',
 '나라/NNG',
 '사람/NNG',
 '이/VCP',
 '면서/ECE',
 '짱/MAG',
 '깨/NNG',
 '를/JKO',
 '수억/NR',
 '이나/JX',
 '죽이/VV',
 '고/ECE',
 '굶기/VV',
 '고/ECE',
 '탄압/NNG',
 '하/XSV',
 '었/EPT',
 '다/ECS',
 '\\/SW',
 'n/OL',
 '세상/NNG',
 '에서/JKM',
 '짱꺠/UN',
 '를/JKO',
 '가장/MAG',
 '많이/MAG',
 '죽이/VV',
 'ㄴ/ETD',
 '사람/NNG',
 '이/JKS',
 '누/NNG',
 '군지/UN',
 '알/VV',
 '는가/EFQ',
 '?/SF',
 '바로/MAG',
 '짱/NNG',
 '꺠/UN',
 '이/VCP',
 'ㄴ/ETD',
 '모택동/NNG',
 '이/VCP',
 '다/ECS',
 '\\/SW',
 'n/OL',
 '짱/NNG',
 '깨/NNG',
 '하/VV',
 '고는/ECE',
 '상종/NNG',
 '을/JKO',
 '말자/NNG',
 '\\/SW',
 'n/OL',
 '짱/NNG',
 '깨/NNG',
 '들/XSN',
 '은/JX',
 '기술/NNG',
 '도/JX',
 '후/NNG',
 '달리/VV',
 '어서/ECD',
 '맨날/MAG',
 '미국/NNP',
 '끄/VV',
 '어/ECS',
 '뺏기/VV',
 '어/ECS',


In [75]:
expressive_type=pd.read_csv('lexicon\expressive-type.csv')

In [76]:
expressive_type

,ngram,freq,dir-action,dir-explicit,dir-speech,indirect,writing-device,max.value,max.prop
0,가*/JKS,1,0.0,0.000000,1.000000,0.000000,0.000000,dir-speech,1.000000
1,가*/VV,3,0.0,0.333333,0.333333,0.333333,0.000000,dir-explicit,0.333333
2,가/JKC,17,0.0,0.411765,0.411765,0.176471,0.000000,dir-explicit,0.411765
3,가/JKS,112,0.0,0.375000,0.312500,0.285714,0.026786,dir-explicit,0.375000
4,가/VV,11,0.0,0.000000,0.818182,0.090909,0.090909,dir-speech,0.818182
5,가/VX,9,0.0,0.000000,0.333333,0.555556,0.111111,indirect,0.555556
6,가격/NNG,3,0.0,0.000000,0.000000,1.000000,0.000000,indirect,1.000000
7,가까이/MAG,2,0.0,1.000000,0.000000,0.000000,0.000000,dir-explicit,1.000000
8,가깝/VA,2,0.0,0.500000,0.000000,0.500000,0.000000,dir-explicit,0.500000
9,가꾸/VV,1,0.0,1.000000,0.000000,0.000000,0.000000,dir-explicit,1.000000


In [77]:
intensity=pd.read_csv('lexicon\intensity.csv')

In [78]:
intensity

,ngram,freq,High,Low,Medium,None,max.value,max.prop
0,가*/JKS,1,0.000000,0.000000,1.000000,0.000000,Medium,1.000000
1,가*/VV,3,0.000000,0.333333,0.666667,0.000000,Medium,0.666667
2,가/JKC,17,0.176471,0.235294,0.588235,0.000000,Medium,0.588235
3,가/JKS,112,0.160714,0.107143,0.696429,0.035714,Medium,0.696429
4,가/VV,11,0.090909,0.181818,0.636364,0.090909,Medium,0.636364
5,가/VX,9,0.111111,0.111111,0.666667,0.111111,Medium,0.666667
6,가격/NNG,3,0.000000,0.000000,1.000000,0.000000,Medium,1.000000
7,가까이/MAG,2,0.000000,0.500000,0.500000,0.000000,Low,0.500000
8,가깝/VA,2,0.000000,0.500000,0.500000,0.000000,Low,0.500000
9,가꾸/VV,1,0.000000,0.000000,1.000000,0.000000,Medium,1.000000


In [79]:
nested_order=pd.read_csv('C:\\Users\\hyunho\\Shiny\\study\\3-1\\tave\\project\\lexicon\\nested-order.csv')

In [80]:
nested_order

,ngram,freq,0,1,2,3,max.value,max.prop
0,가*/JKS,1,0.000000,1.000000,0.000000,0.0,1,1.000000
1,가*/VV,3,0.666667,0.333333,0.000000,0.0,0,0.666667
2,가/JKC,17,0.352941,0.588235,0.058824,0.0,1,0.588235
3,가/JKS,112,0.446429,0.553571,0.000000,0.0,1,0.553571
4,가/VV,11,0.272727,0.727273,0.000000,0.0,1,0.727273
5,가/VX,9,0.555556,0.444444,0.000000,0.0,0,0.555556
6,가격/NNG,3,0.333333,0.666667,0.000000,0.0,1,0.666667
7,가까이/MAG,2,0.500000,0.500000,0.000000,0.0,0,0.500000
8,가깝/VA,2,0.500000,0.500000,0.000000,0.0,0,0.500000
9,가꾸/VV,1,1.000000,0.000000,0.000000,0.0,0,1.000000


In [81]:
subjectivity_polarity=pd.read_csv('C:\\Users\\hyunho\\Shiny\\study\\3-1\\tave\\project\\lexicon\\subjectivity-polarity.csv')

In [82]:
subjectivity_polarity

,ngram,freq,COMP,NEG,NEUT,POS,max.value,max.prop
0,가*/JKS,1,0.0,0.000000,0.000000,1.000000,POS,1.000000
1,가*/JKS;있/VV,1,0.0,0.000000,0.000000,1.000000,POS,1.000000
2,가*/JKS;있/VV;었/EP,1,0.0,0.000000,0.000000,1.000000,POS,1.000000
3,가*/VV,3,0.0,0.000000,0.000000,1.000000,POS,1.000000
4,가*/VV;ㄴ다*/EF,1,0.0,0.000000,0.000000,1.000000,POS,1.000000
5,가/JKC,17,0.0,0.411765,0.117647,0.470588,POS,0.470588
6,가/JKC;되/VV,11,0.0,0.272727,0.181818,0.545455,POS,0.545455
7,가/JKC;되/VV;ㄴ/ETM,2,0.0,0.000000,1.000000,0.000000,NEUT,1.000000
8,가/JKC;되/VV;ㄹ/ETM,1,0.0,0.000000,0.000000,1.000000,POS,1.000000
9,가/JKC;되/VV;어/EC,2,0.0,0.500000,0.000000,0.500000,NEG,0.500000


In [83]:
subjectivity_type=pd.read_csv('C:\\Users\\hyunho\\Shiny\\study\\3-1\\tave\\project\\lexicon\\subjectivity-type.csv')

In [84]:
subjectivity_type

,ngram,freq,Agreement,Argument,Emotion,Intention,Judgment,Others,Speculation,max.value,max.prop
0,가*/JKS,1,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00,Argument,1.000000
1,가*/VV,3,0.000000,0.333333,0.000000,0.000000,0.666667,0.000000,0.00,Judgment,0.666667
2,가/JKC,17,0.058824,0.352941,0.000000,0.000000,0.588235,0.000000,0.00,Judgment,0.588235
3,가/JKS,112,0.008929,0.330357,0.053571,0.008929,0.571429,0.026786,0.00,Judgment,0.571429
4,가/VV,11,0.000000,0.727273,0.000000,0.000000,0.181818,0.090909,0.00,Argument,0.727273
5,가/VX,9,0.000000,0.333333,0.000000,0.000000,0.555556,0.111111,0.00,Judgment,0.555556
6,가격/NNG,3,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00,Judgment,1.000000
7,가까이/MAG,2,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00,Judgment,1.000000
8,가깝/VA,2,0.000000,0.000000,0.500000,0.000000,0.500000,0.000000,0.00,Emotion,0.500000
9,가꾸/VV,1,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00,Judgment,1.000000


In [85]:
for word in list:
    print(word)

일본/NNG
이/JKS
짱/NNG
깨/NNG
를/JKO
수십만/NR
밖에/JX
죽이/VV
지/ECD
못하/VX
ㄴ/ETD
것/NNB
을/JKO
두/VV
고/ECE
우리/NP
는/JX
나쁘/VA
ㄴ/ETD
짓/NNG
이/VCP
라/ECD
욕하/VV
었/EPT
지만/ECE
\/SW
n/OL
모택동/NNG
은/JX
같/VA
은/ETD
나라/NNG
사람/NNG
이/VCP
면서/ECE
짱/MAG
깨/NNG
를/JKO
수억/NR
이나/JX
죽이/VV
고/ECE
굶기/VV
고/ECE
탄압/NNG
하/XSV
었/EPT
다/ECS
\/SW
n/OL
세상/NNG
에서/JKM
짱꺠/UN
를/JKO
가장/MAG
많이/MAG
죽이/VV
ㄴ/ETD
사람/NNG
이/JKS
누/NNG
군지/UN
알/VV
는가/EFQ
?/SF
바로/MAG
짱/NNG
꺠/UN
이/VCP
ㄴ/ETD
모택동/NNG
이/VCP
다/ECS
\/SW
n/OL
짱/NNG
깨/NNG
하/VV
고는/ECE
상종/NNG
을/JKO
말자/NNG
\/SW
n/OL
짱/NNG
깨/NNG
들/XSN
은/JX
기술/NNG
도/JX
후/NNG
달리/VV
어서/ECD
맨날/MAG
미국/NNP
끄/VV
어/ECS
뺏기/VV
어/ECS
다/MAG
쓰/VV
ㄴ다/ECS
그리하/VV
여도/ECD
좀/MAG
모자르/VV
다/ECS
는/JX
평/NNG
을/JKO
듣/VV
는/EPT
다/ECS
\/SW
n/OL
소련/NNG
도/JX
망하/VV
었/EPT
다/ECS
근데/MAG
짱/NNG
깨/NNG
같/VA
은/ETD
클라스/UN
가/JKS
대체/MAG
어/VV
어/ECS
디/VV
ㄹ/ETD
보/VV
아서/ECD
멸족/NNG
을/JKO
면하/VV
ㄹ/ETD
클라/NNG
스인/UN
가/NNG
\/SW
n/OL
공산당/NNG
이/JKS
제일/NNG
통치/NNG
기구/NNG
이/VCP
ㄴ/ETD
나라/NNG
가/JKS
뭐/NP
가/JKS
두렵/VA
나/EFQ
\/SW
n/OL
짱/NNG
깨/NNG
돈/NNG
을/JKO
탐하/VV
다가/ECD
면/NNG
하

In [86]:
polarity["ngram"]

0                     가*/JKS
1                가*/JKS;있/VV
2           가*/JKS;있/VV;었/EP
3                      가*/VV
4               가*/VV;ㄴ다*/EF
5                      가/JKC
6                 가/JKC;되/VV
7           가/JKC;되/VV;ㄴ/ETM
8           가/JKC;되/VV;ㄹ/ETM
9            가/JKC;되/VV;어/EC
10        가/JKC;되/VV;어야지요/EF
11              가/JKC;아니/VCN
12        가/JKC;아니/VCN;ㄴ가/EC
13         가/JKC;아니/VCN;면/EC
14       가/JKC;아니/VCN;ㅂ니다/EF
15                     가/JKS
16              가/JKS;가능/NNG
17        가/JKS;가능/NNG;하/XSA
18              가/JKS;가장/MAG
19       가/JKS;가장/MAG;많이/MAG
20        가/JKS;가장/MAG;어렵/VA
21                가/JKS;굵/VA
22                가/JKS;그/MM
23         가/JKS;그/MM;동안/NNG
24              가/JKS;그리/MAG
25         가/JKS;그리/MAG;밝/VA
26              가/JKS;극대/NNG
27        가/JKS;극대/NNG;화/XSN
28               가/JKS;꺾이/VV
29               가/JKS;꽤/MAG
                ...         
16332                  희박/XR
16333                 희생/NNG
16334           희생/NNG;하/XSV
16335         

In [94]:
polarity[0:100]

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
0,가*/JKS,1,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
1,가*/JKS;있/VV,1,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
2,가*/JKS;있/VV;었/EP,1,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
3,가*/VV,3,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
4,가*/VV;ㄴ다*/EF,1,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
5,가/JKC,17,0.000000,0.470588,0.235294,0.000000,0.294118,NEG,0.470588
6,가/JKC;되/VV,11,0.000000,0.363636,0.272727,0.000000,0.363636,NEG,0.363636
7,가/JKC;되/VV;ㄴ/ETM,2,0.000000,0.000000,1.000000,0.000000,0.000000,NEUT,1.000000
8,가/JKC;되/VV;ㄹ/ETM,1,0.000000,0.000000,1.000000,0.000000,0.000000,NEUT,1.000000
9,가/JKC;되/VV;어/EC,2,0.000000,1.000000,0.000000,0.000000,0.000000,NEG,1.000000


In [112]:
polarity.loc[polarity["ngram"]=='[농구/NNG']

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop


In [88]:
polarity.loc[polarity["ngram"]==list[9]]

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
6238,못하/VX,29,0.0,0.862069,0.034483,0.034483,0.068966,NEG,0.862069


In [89]:
#어절 n-gram 분석
#sentence: 분석할 문장, num_gram: n-gram 단위
def word_ngram(sentence, num_gram):
  # in the case a file is given, remove escape characters
  sentence = sentence.replace('\n', ' ').replace('\r', ' ')
  text = tuple(sentence.split(' '))
  ngrams = [text[x:x+num_gram] for x in range(0, len(text))]
  return tuple(ngrams)
 
#음절 n-gram 분석
#sentence: 분석할 문장, num_gram: n-gram 단위
def phoneme_ngram(sentence, num_gram):
  text = tuple(sentence) # split the sentence into an array of characters
  ngrams = [text[x:x+num_gram] for x in range(0, len(text))]
  return ngrams
 
#n-gram 빈도 리스트 생성
def make_freqlist(ngrams):
  freqlist = {}
 
  for ngram in ngrams:
    if (ngram in freqlist):
      freqlist[ngram] += 1
    else:
      freqlist[ngram] = 1
   
  return freqlist

In [90]:
from operator import itemgetter

In [91]:
ngrams = word_ngram(text, 3)
freqlist = make_freqlist(ngrams)
sorted_freqlist = sorted(freqlist.items(), key=itemgetter(1))
print(sorted_freqlist)

[(('일본이', '짱깨를', '수십만밖에'), 1), (('짱깨를', '수십만밖에', '죽이지'), 1), (('수십만밖에', '죽이지', '못한'), 1), (('죽이지', '못한', '것을'), 1), (('못한', '것을', '두고'), 1), (('것을', '두고', '우리는'), 1), (('두고', '우리는', '나쁜'), 1), (('우리는', '나쁜', '짓이라'), 1), (('나쁜', '짓이라', '욕했지만\\n모택동은'), 1), (('짓이라', '욕했지만\\n모택동은', '같은'), 1), (('욕했지만\\n모택동은', '같은', '나라'), 1), (('같은', '나라', '사람이면서'), 1), (('나라', '사람이면서', '짱깨를'), 1), (('사람이면서', '짱깨를', '수억이나'), 1), (('짱깨를', '수억이나', '죽이고'), 1), (('수억이나', '죽이고', '굶기고'), 1), (('죽이고', '굶기고', '탄압했다\\n세상에서'), 1), (('굶기고', '탄압했다\\n세상에서', '짱꺠를'), 1), (('탄압했다\\n세상에서', '짱꺠를', '가장'), 1), (('짱꺠를', '가장', '많이'), 1), (('가장', '많이', '죽인'), 1), (('많이', '죽인', '사람이'), 1), (('죽인', '사람이', '누군지'), 1), (('사람이', '누군지', '아는가?'), 1), (('누군지', '아는가?', '바로'), 1), (('아는가?', '바로', '짱꺠인'), 1), (('바로', '짱꺠인', '모택동이다\\n짱깨'), 1), (('짱꺠인', '모택동이다\\n짱깨', '하고는'), 1), (('모택동이다\\n짱깨', '하고는', '상종을'), 1), (('하고는', '상종을', '말자\\n짱깨들은'), 1), (('상종을', '말자\\n짱깨들은', '기술도'), 1), (('말자\\n짱깨들은', '기술도', '후달려서'), 1), (('기술도', '후달려서', '맨날'), 1),

In [92]:
# ngram 123 다써서 polarity랑 비교해야함 *처럼 정규표현식은 어떻게할지..